In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import glob
from pydantic import BaseModel
from tqdm.auto import tqdm
from datetime import datetime
import time
from openai import OpenAI
client = OpenAI(api_key="자신의 키를 입력하세요.")

In [ ]:
# data_0913_조정현.csv 라는 파일을 pd의 데이터프레임에 옮기기
df = pd.read_csv('data_0913_조정현.csv')
print("원래 df 개수:",len(df))

# df의 category 열 중 '분류되지 않음'이라는 데이터를 삭제하기
df = df[df['category'] != '분류되지 않음']
print("나중 df 개수:", len(df))

# df의 category 열 중 '역사', '생활', '사회문화', '과학', '수학', '언어'가 아닌 경우를 삭제하기
df = df[df['category'].isin(['역사', '생활', '사회문화', '과학', '수학', '언어'])]
print("나중 df 개수:", len(df))

# 각 카테고리에 해당하는 행 개수를 프린트 하기
print("역사 개수:", len(df[df['category'] == '역사']))
print("생활 개수:", len(df[df['category'] == '생활']))
print("사회문화 개수:", len(df[df['category'] == '사회문화']))
print("과학 개수:", len(df[df['category'] == '과학']))
print("수학 개수:", len(df[df['category'] == '수학']))
print("언어 개수:", len(df[df['category'] == '언어']))

# df의 열 중 source, 담당자 열 삭제하기
df = df.drop(columns=['source', '담당자'])
# df

원래 df 개수: 1000
나중 df 개수: 960
나중 df 개수: 947
역사 개수: 138
생활 개수: 98
사회문화 개수: 218
과학 개수: 408
수학 개수: 8
언어 개수: 77


In [ ]:
# df를 순서대로 100개씩 slice해서 list에 담아두기

# 데이터프레임을 100개씩 자르기
df_list = [df[i:i+100].copy() for i in range(0, len(df), 100)]  # .copy() 추가
print(f"생성된 데이터프레임의 개수: {len(df_list)}")
print(f"첫 번째 배치의 행 수: {len(df_list[0])}")

생성된 데이터프레임의 개수: 10
첫 번째 배치의 행 수: 100


In [ ]:
from pydantic import BaseModel

class Scoring(BaseModel):
    korean_score: int
    data_score: int
    category_score: int
    reason: str

In [ ]:
def evaluate_quality(row):
    """각 행의 품질을 평가하고 점수를 반환합니다."""
    prompt = f"""아래의 Instruction, Output, Category 세트의 품질을 1에서 10 사이의 점수로 평가해주세요.
    한국어 문장이 어색한지도 1에서 10 사이의 점수로 평가해주세요.
    Category가 잘 분류되었는지도 1에서 10 사이의 점수로 평가해주세요.
    1은 매우 낮은 품질, 10은 매우 높은 품질을 의미합니다.
    점수를 매긴 이유는 짧게 작성해주세요.

    Instruction: {row['instruction']}
    Output: {row['output']}
    Category: {row['category']}

    한국어 문법 품질 점수 (1-10):
    품질 점수 (1-10):
    카테고리 분류 점수 (1-10):
    점수를 매긴 이유:"""

    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",  # 또는 다른 적절한 모델
            temperature=0.1,
            messages=[
                {"role": "system", "content": "너는 까다로운 성격을 가진 한국인이야. 엄격한 평가를 해줘."},
                {"role": "user", "content": prompt}
            ],
            response_format=Scoring,
        )

        # 응답에서 점수와 이유를 안전하게 추출
        if response.choices and response.choices[0].message.parsed:
            korean_score = response.choices[0].message.parsed.korean_score
            data_score = response.choices[0].message.parsed.data_score
            category_score = response.choices[0].message.parsed.category_score
            reason = response.choices[0].message.parsed.reason
            return korean_score, data_score, category_score, reason
        else:
            return 5, 5, 5, 'Invalid response structure'  # 응답 구조가 올바르지 않은 경우 기본값 반환
    except Exception as e:
        print(f"Error evaluating row: {e}")
        return 5, 5, 5, 'Error occurred'  # 오류 발생 시 기본값 반환

In [ ]:
from tqdm.auto import tqdm
from datetime import datetime

# 전체 진행 상황을 표시할 tqdm 객체 생성
overall_progress = tqdm(df_list, desc="전체 배치 처리", position=0)

# 각 배치를 개별적으로 처리
for batch_index, batch in enumerate(overall_progress):
    # 결과를 저장할 새로운 열 추가
    batch['korean_score'] = 0
    batch['data_score'] = 0
    batch['category_score'] = 0
    batch['quality_reason'] = ''

    # 현재 배치의 진행 상황을 표시할 tqdm 객체 생성
    batch_progress = tqdm(total=len(batch), desc=f"배치 {batch_index+1} 처리", position=1, leave=False)

    # 각 행에 대해 품질 평가 수행
    for index, row in batch.iterrows():
        korean_score, data_score, category_score, reason = evaluate_quality(row)
        batch.at[index, 'korean_score'] = korean_score
        batch.at[index, 'data_score'] = data_score
        batch.at[index, 'category_score'] = category_score
        batch.at[index, 'quality_reason'] = reason

        # 내부 진행 막대 업데이트
        batch_progress.update(1)

    # 내부 진행 막대 종료
    batch_progress.close()

    # 현재 시간을 가져옵니다
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 파일 이름에 배치 번호와 시간을 포함시킵니다
    filename = f'evaluated_data_batch_{batch_index+1}_{current_time}.csv'

    # 결과를 CSV 파일로 저장
    batch.to_csv(filename, index=False)

    # 처리 간 잠시 대기 (필요한 경우)
    time.sleep(1)

print("\n모든 배치 처리가 완료되었습니다.")

전체 배치 처리:   0%|          | 0/10 [00:00<?, ?it/s]

배치 1 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 2 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 3 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 4 처리:   0%|          | 0/100 [00:00<?, ?it/s]

...

배치 7 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 8 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 9 처리:   0%|          | 0/100 [00:00<?, ?it/s]

배치 10 처리:   0%|          | 0/60 [00:00<?, ?it/s]


모든 배치 처리가 완료되었습니다.


In [ ]:
import glob

# CSV 파일들의 경로 패턴
path = 'evaluated_data_batch_*.csv'

# 모든 CSV 파일의 리스트를 가져옵니다
all_files = glob.glob(path)

# 결과를 저장할 빈 리스트를 생성합니다
df_list = []

# tqdm으로 진행 상황을 표시하면서 각 파일을 읽어 리스트에 추가합니다
for filename in tqdm(all_files, desc="파일 읽는 중"):
    df = pd.read_csv(filename)
    df_list.append(df)

# 모든 데이터프레임을 하나로 합칩니다
combined_df = pd.concat(df_list, ignore_index=True)

# 결과를 새로운 CSV 파일로 저장합니다
output_filename = 'combined_evaluated_data.csv'
combined_df.to_csv(output_filename, index=False)

print(f"모든 파일이 성공적으로 합쳐져 {output_filename}로 저장되었습니다.")
print(f"총 {len(combined_df)} 행의 데이터가 처리되었습니다.")

파일 읽는 중:   0%|          | 0/10 [00:00<?, ?it/s]

모든 파일이 성공적으로 합쳐져 combined_evaluated_data.csv로 저장되었습니다.
총 960 행의 데이터가 처리되었습니다.


In [ ]:
# CSV 파일 읽기
print("CSV 파일을 읽는 중...")
df = pd.read_csv('combined_evaluated_data.csv')

# df의 category 열 중 '역사', '생활', '사회문화', '과학', '수학', '언어'가 아닌 경우를 삭제하기
df = df[df['category'].isin(['역사', '생활', '사회문화', '과학', '수학', '언어'])]

# 전체 행 수 저장
total_rows = len(df)

# 점수가 cut 이하인 행 카운트

korean_cut = 7
data_cut = 6
category_cut = 5

print("일정 점수 이하 점수 행 카운트 중...")
low_korean_score = sum(df['korean_score'] <= korean_cut)
low_data_score = sum(df['data_score'] <= data_cut)
low_category_score = sum(df['category_score'] <= category_cut)

# 점수가 cut 초과인 행만 필터링
print("커트 초과 점수 행 필터링 중...")
filtered_df = df[(df['korean_score'] > korean_cut) & (df['data_score'] > data_cut) & (df['category_score'] > category_cut)]

# 카테고리 '과학' 중 data_score이 7점 초과, category_score이 7점 초과인 행 필터링
filtered_df = filtered_df[(filtered_df['category'] != '과학') | ((filtered_df['data_score'] > 6) & (filtered_df['category_score'] > 7))]

# 필터링 후 행 수
filtered_rows = len(filtered_df)

# 결과 출력
print(f"\n결과:")
print(f"전체 행 수: {total_rows}")
print(f"Korean 점수 {korean_cut}점 이하 행 수: {low_korean_score}")
print(f"Data 점수 {data_cut}점 이하 행 수: {low_data_score}")
print(f"Category 점수 {category_cut}점 이하 행 수: {low_category_score}")
print(f"삭제된 행 수: {total_rows - filtered_rows}")
print(f"남은 행 수: {filtered_rows}")
print("-"*20, "필터 전", "-"*20)
# 각 카테고리에 해당하는 행 개수를 프린트 하기
print("역사 개수:", len(df[df['category'] == '역사']))
print("생활 개수:", len(df[df['category'] == '생활']))
print("사회문화 개수:", len(df[df['category'] == '사회문화']))
print("과학 개수:", len(df[df['category'] == '과학']))
print("수학 개수:", len(df[df['category'] == '수학']))
print("언어 개수:", len(df[df['category'] == '언어']))
# 각 카테고리에 해당하는 행 개수를 프린트 하기
print("-"*20, "필터 후", "-"*20)
print("역사 개수:", len(filtered_df[filtered_df['category'] == '역사']))
print("생활 개수:", len(filtered_df[filtered_df['category'] == '생활']))
print("사회문화 개수:", len(filtered_df[filtered_df['category'] == '사회문화']))
print("과학 개수:", len(filtered_df[filtered_df['category'] == '과학']))
print("수학 개수:", len(filtered_df[filtered_df['category'] == '수학']))
print("언어 개수:", len(filtered_df[filtered_df['category'] == '언어']))

# 필터링된 데이터를 새 CSV 파일로 저장
output_filename = f'k{korean_cut}_d{data_cut}_c{category_cut}_filtered_evaluated_data.csv'
filtered_df.to_csv(output_filename, index=False)
print(f"\n필터링된 데이터가 {output_filename}로 저장되었습니다.")

CSV 파일을 읽는 중...
일정 점수 이하 점수 행 카운트 중...
커트 초과 점수 행 필터링 중...

결과:
전체 행 수: 947
Korean 점수 7점 이하 행 수: 363
Data 점수 6점 이하 행 수: 247
Category 점수 5점 이하 행 수: 132
삭제된 행 수: 480
남은 행 수: 467
-------------------- 필터 전 --------------------
역사 개수: 138
생활 개수: 98
사회문화 개수: 218
과학 개수: 408
수학 개수: 8
언어 개수: 77
-------------------- 필터 후 --------------------
역사 개수: 77
생활 개수: 51
사회문화 개수: 116
과학 개수: 191
수학 개수: 4
언어 개수: 28

필터링된 데이터가 k7_d6_c5_filtered_evaluated_data.csv로 저장되었습니다.


In [ ]:
# output_csv -> output_xlsx

df = pd.read_csv(output_filename)
df.to_excel(output_filename.replace('.csv', '.xlsx'), index=False)

# 저장 알리미
print(f"{output_filename.replace('.csv', '.xlsx')}로 저장되었습니다.")

k7_d6_c5_filtered_evaluated_data.xlsx로 저장되었습니다.


In [ ]:
# korean_score, data_score, category_score, quality_reason 열 삭제
df = df.drop(columns=['korean_score', 'data_score', 'category_score', 'quality_reason'])
# 모든 행에 source = "korean_rlhf_dataset", 담당자 = "조정현"으로 되어있는 열을 추가
df['source'] = "korean_rlhf_dataset"
df['담당자'] = "조정현"
df.to_csv('final_evaluated_data.csv', index=False)
df.to_excel('final_evaluated_data.xlsx', index=False)

# 저장 알리미
print("final_evaluated_data.csv로 저장되었습니다.")

final_evaluated_data.csv로 저장되었습니다.


In [ ]:
# combined_evaluated_data.csv -> .xlsx 로 바꿔서 저장

df = pd.read_csv('combined_evaluated_data.csv')
df.to_excel('combined_evaluated_data.xlsx', index=False)

# 저장 알리미
print(f"combined_evaluated_data.xlsx로 저장되었습니다.")